In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle

In [2]:
def one_hot(y, num_classes):
  """
  Đầu vào:
    y:
      Dạng: numpy array
      Miêu tả: Các giá trị nhãn
      Chiều: (batch_size,)
    num_classes:
      Dạng: Python integer
      Miêu tả: Số lượng nhãn
      Điều kiện: num_classes > 0
      Ví dụ: 10
  Đầu ra:
    y_one_hot:
      Miêu tả: Trả về ma trận các vector one hot của từng nhãn
      Chiều: (batch_size, num_classes)
  """
  y_one_hot = np.squeeze(np.eye(num_classes)[y.reshape(-1)])

  return y_one_hot

def flatten_images(images):
  """
  Thực thi duỗi ảnh từ (batch_size, width, height) thành (batch_size, width x height)
  Đầu vào:
    images:
      Dạng: numpy array
      Miêu tả: Ma trận các ảnh
      Chiều: (batch_size, width, height)
      Ví dụ: (32, 28, 28)
  Đầu ra:
    flattened_images
      Miêu tả: ma trận trong đó các ảnh được duỗi thành vector
      Chiều: (batch_size, image_vector_size) = (batch_size, width x height)
      Ví dụ: (32, 28 x 28) = (32, 784)
  """
  flattened_images = np.reshape(images, (images.shape[0], -1))

  return flattened_images

def normalize_images(images):
  """
  Hàm chuẩn hóa ảnh các giá trị pixel để nằm trong khoảng từ 0 đến 1
  Đầu vào:
    images:
      Dạng: numpy array
      Miêu tả: Ma trận các vector ảnh
      Chiều: (batch_size, image_vector_size)
      Ví dụ: (32, 784)
  Đầu ra:
    normailized_images
      Miêu tả: ma trận trong đó các vector ảnh được chuẩn hóa
      Chiều: (batch_size, image_vector_size)
      Ví dụ: (32, 784)
  """
  normailized_images = images / 255.0

  return normailized_images

In [3]:
class Relu:
  def __init__(self):
    pass

  def Z_grad(self, Z):
    Z_grad = np.zeros_like(Z)
    Z_grad[Z > 0] = 1
    return Z_grad

  def forward(self, Z):
    self.Z = Z
    A = np.maximum(0,Z)
    return A, Z

  def backward(self, dA, cache):
    return dA * self.Z_grad(cache)

In [4]:
class Softmax:
  def __init__(self):
    pass

  def forward(self,Z):
    e_Z = np.exp(Z)
    self.Y_hat = e_Z/ e_Z.sum(axis=0)
    return self.Y_hat, Z

  def backward(self, Y_train_one_hot, Y_hat):
    return Y_hat - Y_train_one_hot

In [5]:
class Loss:
  def __init__(self):
    pass
  def cal(self, Y_hat, Y):
    m = Y.shape[1]
    cost = self.forward(Y_hat, Y) / m
    return cost

class Crossentropy(Loss):
  def forward(self, Y_hat, Y):
    correct = - np.sum(np.sum(Y * np.log(Y_hat), 0))
    return correct

In [6]:
class Linear:
  def __init__(self, num_in, num_neuron):
    self.W = np.random.randn(num_neuron, num_in) * 0.1 # khởi tạo ma trận trọng số chiều (chiều Z đầu ra, chiều A đầu vào)
    self.b = np.zeros((num_neuron, 1)) # khởi tạo vector bias chiều (Z đầu ra)

  def linear_forward(self, A_pre): #A_pre là ma trận (chiều vector dữ liệu, số điểm dữ liệu)
    Z = np.dot(self.W, A_pre) + self.b
    linear_cache = (A_pre, self.W, self.b)
    return Z, linear_cache

  def linear_backward(self, dZ, A_pre):
    m = dZ.shape[1]
    dW = dZ.dot(A_pre.T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    dA_prev = self.W.T.dot(dZ)
    return dA_prev, dW, db

  def forward(self, A_pre, activation):
    self.activation = activation
    #Nhân tuyến tính
    Z, linear_cache = self.linear_forward(A_pre)

    #phi tuyến
    if self.activation == "relu":
      A, activation_cache = Relu().forward(Z)

    elif self.activation == "softmax":
      A, activation_cache = Softmax().forward(Z)

    self.cache = (linear_cache, activation_cache)
    return A

  def backward(self, residual_in, Y_hat, learning_rate):
    linear_cache, activation_cache = self.cache #cache ở đây là (linear_cache, activation_cache), linear_cache gồm (A_pre, W, b), activation_cache gồm (Z)
    A_pre, W, b = linear_cache

    if self.activation == "relu":
      dA = residual_in
      dZ = Relu().backward(dA, activation_cache)
    elif self.activation == "softmax":
      Y_train_one_hot = residual_in
      dZ = Softmax().backward(Y_train_one_hot, Y_hat)

    dA_prev, dW, db = self.linear_backward(dZ, A_pre)

    self.W = self.update_parameters(self.W, dW, learning_rate)# cập nhật W
    self.b = self.update_parameters(self.b, db, learning_rate)# cập nhật b
    return dA_prev

  def update_parameters(self, parameters, grads, learning_rate):
    parameters -= (grads * learning_rate)
    return parameters


In [7]:
class Net:
  def __init__(self):
    self.layers = []
    self.activations = []
    self.costs = []

  def addlayer(self, layer, activation = "relu"):
    self.layers.append(layer)
    self.activations.append(activation)

  def fit(self, X_train, Y_train, X_val, Y_val, epoch=250, learning_rate = 0.01):
    X_train = normalize_images(X_train)
    X_val = normalize_images(X_val)
    X_train = flatten_images(X_train).T
    X_val = flatten_images(X_val).T
    Y_train_one_hot = one_hot(Y_train, 10).T

    for i in range(epoch):
      cost = self.train(X_train, Y_train_one_hot, learning_rate)
      self.costs.append(cost)
      train_acc = self.acc(X_train, Y_train)
      val_acc = self.acc(X_val, Y_val)
      print("Epoch {}: Cost: {:.3f} Acc: {:.3f} Validation Acc: {:.3f}".format(i + 1, np.squeeze(cost), train_acc, val_acc))

  def train(self, X, Y_one_hot, learning_rate):
    layer_nums = len(self.layers)
    in_data = X

    #forward
    for i in range(layer_nums):
      out_data = self.layers[i].forward(in_data, self.activations[i]) #cache ở đây là (linear_cache, activation_cache), linear_cache gồm (A_pre, W, b), activation_cache gồm (Z)
      in_data = out_data
    Y_hat = out_data
    #tính loss
    loss = Crossentropy()
    cost = loss.cal(Y_hat, Y_one_hot)

    #backward
    in_residual = Y_one_hot
    for i in range(layer_nums-1, -1, -1):
      dA = self.layers[i].backward(in_residual, Y_hat, learning_rate)
      in_residual = dA
    return cost

  def acc(self, X, Y):
    layer_nums = len(self.layers)
    in_data = X
    for i in range(len(self.layers)):
      out_data = self.layers[i].forward(in_data, self.activations[i])
      in_data = out_data

    Y_hat = np.argmax(out_data, axis=0)
    acc = np.sum(Y_hat == Y) / len(Y)
    self.outacc = acc
    return acc

Training


In [8]:
import tensorflow as tf
(X_train, Y_train), (X_val, Y_val) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [9]:
num_of_label = len(set(Y_train))
shape_of_in_data = X_train.shape[1]

In [10]:
model = Net()
model.addlayer(Linear(784, 128))
model.addlayer(Linear(128, 256))
model.addlayer(Linear(256, 64))
model.addlayer(Linear(64, 10), "softmax")

In [11]:
model.fit(X_train, Y_train, X_val, Y_val, 400, learning_rate = 0.15)

Epoch 1: Cost: 2.409 Acc: 0.170 Validation Acc: 0.169
Epoch 2: Cost: 2.248 Acc: 0.256 Validation Acc: 0.258
Epoch 3: Cost: 2.187 Acc: 0.327 Validation Acc: 0.334
Epoch 4: Cost: 2.132 Acc: 0.383 Validation Acc: 0.387
Epoch 5: Cost: 2.076 Acc: 0.423 Validation Acc: 0.429
Epoch 6: Cost: 2.018 Acc: 0.456 Validation Acc: 0.462
Epoch 7: Cost: 1.956 Acc: 0.486 Validation Acc: 0.492
Epoch 8: Cost: 1.888 Acc: 0.517 Validation Acc: 0.521
Epoch 9: Cost: 1.816 Acc: 0.547 Validation Acc: 0.553
Epoch 10: Cost: 1.740 Acc: 0.579 Validation Acc: 0.584
Epoch 11: Cost: 1.659 Acc: 0.609 Validation Acc: 0.616
Epoch 12: Cost: 1.577 Acc: 0.637 Validation Acc: 0.644
Epoch 13: Cost: 1.494 Acc: 0.661 Validation Acc: 0.671
Epoch 14: Cost: 1.412 Acc: 0.684 Validation Acc: 0.690
Epoch 15: Cost: 1.333 Acc: 0.702 Validation Acc: 0.708
Epoch 16: Cost: 1.257 Acc: 0.718 Validation Acc: 0.724
Epoch 17: Cost: 1.186 Acc: 0.732 Validation Acc: 0.741
Epoch 18: Cost: 1.120 Acc: 0.744 Validation Acc: 0.752
Epoch 19: Cost: 1.0